### Selenium 사용 동적 크롤링 - 네이버 환율

In [1]:
# Selenium 설치.
# !pip install selenium

In [2]:
import pandas as pd
import bs4
from selenium import webdriver

#### 사이트에 들어가서 탐색해 본다:

1. 먼저 다음 사이트에 들어가서 살펴본다:  [네이버 금융](https://finance.naver.com/marketindex/)
2. 크롬에서 3점 메뉴 버튼을 누르고 **도구 더보기** ==> **개발자 도구**를 선택해서 개발자 도구를 연다.
3. 개발자 도구 왼쪽 상단의 화살표 버튼을 눌러서 사이트 일부를 서택하고 해당 HTML코드를 볼 수 있다. "[CTRL] + [SHIFT] + C" 조합으로 대체할 수 있다.

#### 이제는 Selenium을 사용해 본다:

In [3]:
import os
os.getcwd()

'C:\\workspace\\Python\\4-2) 핀테크 교육\\Textbook2\\4.Data_collection_and_processing\\notebook'

In [4]:
# 웹드라이버 로딩.
my_driver = webdriver.Chrome('chromedriver.exe') 

In [5]:
# 사이트에 접속.
my_driver.get("https://finance.naver.com/marketindex")

In [6]:
# CSS 셀렉터를 사용하여 요소 가져오기. 
res = my_driver.find_elements_by_css_selector('span.value')

In [7]:
# 속성을 직접 가져온다. BeautifulSoup 불필요.
rate =[]
for i in range(4):
    rate += [float(''.join((res[i].text).split(',')))]     # 1000 단위 쉼표 "," 제거 처리.
result = {"외환":["USD","JPY","EUR","CNY"], "환율":rate}
df=pd.DataFrame(result)
df

,외환,환율
0,USD,1159.50
1,JPY,1061.28
2,EUR,1288.67
3,CNY,167.46


#### Parsing 부분은 BeautifulSoup로 처리하면 간편하다:

In [8]:
my_html = my_driver.page_source                      # 사이트 내용을 가져온다.
soup = bs4.BeautifulSoup(my_html, 'html.parser')     # BeautifulSoup 객체 반환. 'html.parser' 사용.

In [9]:
rate =[]
for i in range(4):
    rate += [float(''.join((res[i].text).split(',')))]     # 1000 단위 쉼표 "," 제거 처리.
result = {"외환":["USD","JPY","EUR","CNY"], "환율":rate}
df=pd.DataFrame(result)
df

,외환,환율
0,USD,1159.50
1,JPY,1061.28
2,EUR,1288.67
3,CNY,167.46


#### iframe안의 내용 가져오기 (option):

In [10]:
# CSS 셀렉터를 사용하여 요소 가져오기. 
res = my_driver.find_elements_by_tag_name('iframe')

In [11]:
my_html = ''
for x in res:
    try:
        my_driver.switch_to_frame(x)                    # iframe으로 가기.
        my_html += my_driver.page_source + "\n"         # 내용을 이어 붙인다.
        my_driver.switch_to_default_content()           # 복귀.
    except:
        my_driver.switch_to_default_content()           # 오류가 나도 복귀.

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:4: DeprecationWarning: use driver.switch_to.frame instead
  after removing the cwd from sys.path.
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:6: DeprecationWarning: use driver.switch_to.default_content instead
  


In [12]:
# 수집된 iframe 안의 html을 BeautifulSoup 객체로 변환.
soup = bs4.BeautifulSoup(my_html, 'html.parser')     # BeautifulSoup 객체 반환. 'html.parser' 사용.

In [13]:
res1=soup.find('table')
res2=res1.find_all('tr')

In [14]:
n = len(res2)
for i in range(2,n):
    print(res2[i].text.strip())
    print('-'*50 + '\n')

미국 USD
				
				
1,159.50
1,179.79
1,139.21
1,170.80
1,148.20
1.000
--------------------------------------------------

유럽연합 EUR
				
				
1,288.67
1,314.31
1,263.03
1,301.55
1,275.79
1.111
--------------------------------------------------

일본 JPY (100엔)
					
					
					
				
				
1,061.28
1,079.85
1,042.71
1,071.68
1,050.88
0.915
--------------------------------------------------

중국 CNY
				
				
167.46
175.83
159.09
169.13
165.79
0.144
--------------------------------------------------

홍콩 HKD
				
				
149.14
152.07
146.21
150.63
147.65
0.129
--------------------------------------------------

대만 TWD
				
				
38.71
43.78
36.01
0.00
0.00
0.033
--------------------------------------------------

영국 GBP
				
				
1,519.52
1,549.45
1,489.59
1,534.71
1,504.33
1.311
--------------------------------------------------

오만 OMR
				
				
3,011.53
3,279.55
2,830.84
0.00
0.00
2.597
--------------------------------------------------

캐나다 CAD
				
				
889.32
906.83
871.81
898.21
880.43
0.767

#### click()으로 페이지 작동:

In [15]:
# 탭 1 선택. 
res = my_driver.find_element_by_css_selector('span.tab1')
res.click()
print(my_driver.current_url)

https://finance.naver.com/marketindex/?tabSel=exchange#tab_section


In [16]:
# 탭 2 선택. 
res = my_driver.find_element_by_css_selector('span.tab2')
res.click()
print(my_driver.current_url)

https://finance.naver.com/marketindex/?tabSel=worldExchange#tab_section


In [17]:
# 탭 3 선택.  
res = my_driver.find_element_by_css_selector('span.tab3')
res.click()
print(my_driver.current_url)

https://finance.naver.com/marketindex/?tabSel=gold#tab_section


In [18]:
# 탭 4 선택.  
res = my_driver.find_element_by_css_selector('span.tab4')
res.click()
print(my_driver.current_url)

https://finance.naver.com/marketindex/?tabSel=materials#tab_section
